# Hartree-Fock VQE Demo

In [1]:
import tqdm
import pickle

import numpy as np
import scipy as sp
import numpy_ml

from molecular_data import MolecularData, obi_basis_rotation, tbi_basis_rotation, generate_hamiltonian
from QuICT.algorithm.quantum_machine_learning.model.VQE.operators.encoder import JordanWigner
from QuICT.algorithm.quantum_machine_learning.model.VQE import HartreeFockVQENet
from QuICT.algorithm.quantum_machine_learning.utils import Hamiltonian

/home/chenziheng/base/lib/python3.8/site-packages/numpy_ml/rl_models/rl_utils.py:25: DependencyWarning: Agents in `numpy_ml.rl_models` use the OpenAI gym for training. To install the gym environments, run `pip install gym`. For more information, see https://github.com/openai/gym.
  warnings.warn(fstr, DependencyWarning)


Please install quict_sim first, you can use 'pip install quict_sim' to install. 
Please install pytorch, torch-geometric, torch-sparse, tensorboard, cupy and quict_ml first, you can use 'pip install quict-ml' to install quict_ml. 
Please install pytorch, cupy and quict_ml first, you can use 'pip install quict-ml' to install quict_ml. 


## Load the data

In [2]:
moldir = "./molecular_data"
molfile = moldir + "/H6_sto-3g_singlet_linear_r-1.3.hdf5"
moldata = MolecularData(molfile)

overlap = np.load(moldir + "/overlap.npy")
Hcore = np.load(moldir + "/h_core.npy")
two_electron_integral = np.einsum("psqr", np.load(moldir + "/tei.npy"))  # (1, 1, 0, 0)

_, X = sp.linalg.eigh(Hcore, overlap)
obi = obi_basis_rotation(Hcore, X)
tbi = tbi_basis_rotation(two_electron_integral, X)
molecular_hamiltonian = generate_hamiltonian(moldata.nuclear_repulsion, obi, tbi)

## Convert the Hamiltonian

In [3]:
fermi_op = molecular_hamiltonian.get_fermion_operator()
orbitals = 2 * moldata.n_orbitals
electrons = moldata.n_electrons
qubit_op = JordanWigner(orbitals).encode(fermi_op)
hamiltonian = Hamiltonian(qubit_op.to_hamiltonian())

ham_str = pickle.dumps(hamiltonian)
with open('hamiltonian', 'wb') as f:
    f.write(ham_str)

## Calculate the ground energy

In [6]:
MAX_ITERS = 1000
LR = 0.1

orbitals = 2 * moldata.n_orbitals
electrons = moldata.n_electrons
with open('hamiltonian', 'rb') as f:
    hamiltonian = pickle.load(f)
hfvqe_net = HartreeFockVQENet(orbitals, electrons, hamiltonian)
optim = numpy_ml.neural_nets.optimizers.Adam(lr=LR)
energy = []

loader = tqdm.trange(MAX_ITERS, desc="Training", leave=True)
for it in loader:
    state, loss = hfvqe_net.run_step(optim)
    print(hfvqe_net.params)
    for g in hfvqe_net._circuit.gates:
        print(g.type, g.parg)
    loader.set_postfix(loss=loss.item())

print(energy)
print(hfvqe_net.params)
print(float(loss))

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

Training:   0%|          | 1/1000 [00:00<11:53,  1.40it/s, loss=-2.37]

Variable(pargs=[ 0.78999131 -0.08697694  0.97579485 -1.19068494  0.03261784 -2.89210455
  1.09029962  0.07775384  0.33661218 -0.66755274 -1.07452473  1.24851497
  0.15297638 -1.52629125 -0.08618069 -0.20645975  1.00093978  0.70579584
  0.57524683 -2.0491501  -0.71809314  0.06071739 -0.39202209  1.13687894
 -0.70491464 -1.70380561 -1.87509786 -0.74985719  0.21113419  1.43557458
 -0.26906351  1.09579743 -3.35641156  0.55925926 -1.1420985   0.88171877], grads=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], identity=fd79bfd016f111ee8f93f7b57e969e00, shape=(36,))
[<QuICT.core.gate.gate.BasicGate object at 0x7f2e7f105820>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f105640>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f105850>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f105880>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f105790>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f1058b0>, <QuICT.core.gate

Training:   0%|          | 2/1000 [00:01<11:47,  1.41it/s, loss=-2.37]

Variable(pargs=[ 0.78999131 -0.08697694  0.97579485 -1.19068494  0.03261784 -2.89210455
  1.09029962  0.07775384  0.33661218 -0.66755274 -1.07452473  1.24851497
  0.15297638 -1.52629125 -0.08618069 -0.20645975  1.00093978  0.70579584
  0.57524683 -2.0491501  -0.71809314  0.06071739 -0.39202209  1.13687894
 -0.70491464 -1.70380561 -1.87509786 -0.74985719  0.21113419  1.43557458
 -0.26906351  1.09579744 -3.35641156  0.55925926 -1.1420985   0.88171877], grads=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], identity=fd79bfd016f111ee8f93f7b57e969e00, shape=(36,))
[<QuICT.core.gate.gate.BasicGate object at 0x7f2e7f80e3a0>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f80e790>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f80e4c0>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f80e460>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f80e580>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f80e4f0>, <QuICT.core.gate

...

Variable(pargs=[ 0.78999132 -0.08697694  0.97579485 -1.19068494  0.03261785 -2.89210456
  1.09029962  0.07775384  0.33661218 -0.66755274 -1.07452473  1.24851497
  0.15297638 -1.52629125 -0.08618069 -0.20645975  1.00093978  0.70579584
  0.57524683 -2.0491501  -0.71809314  0.06071739 -0.39202209  1.13687895
 -0.70491464 -1.70380561 -1.87509786 -0.74985719  0.21113419  1.43557458
 -0.26906351  1.09579744 -3.35641156  0.55925926 -1.1420985   0.88171877], grads=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], identity=fd79bfd016f111ee8f93f7b57e969e00, shape=(36,))
[<QuICT.core.gate.gate.BasicGate object at 0x7f2eccba6b80>, <QuICT.core.gate.gate.BasicGate object at 0x7f2eccba6a90>, <QuICT.core.gate.gate.BasicGate object at 0x7f2eccba6a00>, <QuICT.core.gate.gate.BasicGate object at 0x7f2eccba6a60>, <QuICT.core.gate.gate.BasicGate object at 0x7f2eccba6f40>, <QuICT.core.gate.gate.BasicGate object at 0x7f2eccba6610>, <QuICT.core.gate

Training:   1%|          | 7/1000 [00:05<12:58,  1.28it/s, loss=-2.37]

Variable(pargs=[ 0.78999132 -0.08697694  0.97579485 -1.19068494  0.03261785 -2.89210456
  1.09029962  0.07775384  0.33661218 -0.66755274 -1.07452473  1.24851497
  0.15297638 -1.52629125 -0.08618069 -0.20645975  1.00093978  0.70579584
  0.57524683 -2.0491501  -0.71809314  0.06071739 -0.39202209  1.13687895
 -0.70491464 -1.70380561 -1.87509786 -0.74985719  0.21113419  1.43557458
 -0.26906351  1.09579744 -3.35641156  0.55925926 -1.1420985   0.88171877], grads=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], identity=fd79bfd016f111ee8f93f7b57e969e00, shape=(36,))
[<QuICT.core.gate.gate.BasicGate object at 0x7f2e7f8f70d0>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f8f73a0>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f8f7fa0>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f8f7c40>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f8f77f0>, <QuICT.core.gate.gate.BasicGate object at 0x7f2e7f8f7df0>, <QuICT.core.gate

Training:   1%|          | 7/1000 [00:06<14:20,  1.15it/s, loss=-2.37]


KeyboardInterrupt: 